In [ ]:
# default_exp solar.nsrdb

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

# National Solar Radiation Database (NSRDB)

> The National Solar Radiation Database (NSRDB) is a serially complete collection of meteorological and solar irradiance data sets for the United States and a growing list of international locations. The data are publicly available at no cost to the user and maintained by NREL. 

## Data Query

Find NSRDB datasets for a location

In [ ]:
#export
from nrel_dev_api._core import get_request, check_api_key

class NSRDB_DataQuery:
    """Returns information on the closest NSRDB datasets for a location 
    including a set of links that can be used to download the data.
    """
    
    QUERY_URL = "/api/solar/nsrdb_data_query.json"
    
    def __init__(self,
                 api_key=None,
                 wkt=None,
                 address=None,
                 lat=None,
                 lon=None,
                 dataset_type=None,
                 show_empty=False,
                ):
        
        if api_key is None:
            api_key = check_api_key()
            
        self._params = {"api_key": api_key}
        
        # if well-known text is not provided look for address or lat/lon
        if not wkt:
            if not address:
                self._params.update({"lat" : lat, "lon" : lon})
            else:
                self._params.update({"address" : address})
        else:
            self._params.update({"wkt" : wkt})
        
        if dataset_type:
            self._params.update({"type" : dataset_type})
            
        if show_empty:
            self._params.update({"show_empty" : show_empty})
        
        r = get_request(self.QUERY_URL, self._params)
        
        # complete response as a dictionary
        self.response = r.json()
        
        self.outputs = self.response["outputs"]
        

In [ ]:
#hide
NREL_API_KEY = "DEMO_KEY"

Let's set our NREL API key.

In [ ]:
import os
from nrel_dev_api import set_nrel_api_key

set_nrel_api_key(os.environ.get("NREL_API_KEY"))

Get information for a single location point using latitude and longitude.

In [ ]:
nsrdb_data_query = NSRDB_DataQuery(lat=21.204, lon=72.839)

# check the outputs
nsrdb_data_query.outputs

[{'apiDocs': 'https://developer.nrel.gov/docs/solar/nsrdb/msg-iodc/',
  'availableIntervals': [15, 30, 60],
  'enabled': True,
  'displayName': 'MSG IODC: PSM v3',
  'metadataLink': 'https://nsrdb.nrel.gov/current-version',
  'name': 'msg-iodc',
  'ranking': 20,
  'availableYears': [2017, 2018, 2019],
  'apiUrl': 'https://developer.nrel.gov/api/nsrdb/v2/solar/msg-iodc-download',
  'links': [{'year': 2017,
    'interval': 15,
    'link': 'https://developer.nrel.gov/api/nsrdb/v2/solar/msg-iodc-download.csv?names=2017&wkt=POINT%2872.839+21.204%29&interval=15&api_key=yourapikey&email=youremail'},
   {'year': 2018,
    'interval': 15,
    'link': 'https://developer.nrel.gov/api/nsrdb/v2/solar/msg-iodc-download.csv?names=2018&wkt=POINT%2872.839+21.204%29&interval=15&api_key=yourapikey&email=youremail'},
   {'year': 2019,
    'interval': 15,
    'link': 'https://developer.nrel.gov/api/nsrdb/v2/solar/msg-iodc-download.csv?names=2019&wkt=POINT%2872.839+21.204%29&interval=15&api_key=yourapikey&e

Using a well-known text (WKT) representation of the geometry for which to extract data

In [ ]:
nsrdb_data_query = NSRDB_DataQuery(wkt="POINT(91.287 23.832)")

# check the outputs
nsrdb_data_query.outputs

[{'apiDocs': 'https://developer.nrel.gov/docs/solar/nsrdb/himawari-download/',
  'availableIntervals': [10, 30, 60],
  'enabled': True,
  'displayName': 'Himawari',
  'metadataLink': 'https://nsrdb.nrel.gov',
  'name': 'himawari',
  'ranking': 20,
  'availableYears': [2016, 2017, 2018, 2019],
  'apiUrl': 'https://developer.nrel.gov/api/nsrdb/v2/solar/himawari-download',
  'links': [{'year': 2016,
    'interval': 10,
    'link': 'https://developer.nrel.gov/api/nsrdb/v2/solar/himawari-download.csv?names=2016&wkt=POINT%2891.287+23.832%29&interval=10&api_key=yourapikey&email=youremail'},
   {'year': 2017,
    'interval': 10,
    'link': 'https://developer.nrel.gov/api/nsrdb/v2/solar/himawari-download.csv?names=2017&wkt=POINT%2891.287+23.832%29&interval=10&api_key=yourapikey&email=youremail'},
   {'year': 2018,
    'interval': 10,
    'link': 'https://developer.nrel.gov/api/nsrdb/v2/solar/himawari-download.csv?names=2018&wkt=POINT%2891.287+23.832%29&interval=10&api_key=yourapikey&email=your

## Data Downloader

> Download solar irradiance and meteorological data (including spectral data) for a location